In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
from intvalpy import lineqs
import sympy
import numpy as np
import os
import json
from scipy.optimize import linprog
from ipywidgets import interact, widgets
from functools import lru_cache
#plt.rcParams['text.usetex'] = True

# Mixed Integer Linear Program (MILP)
$$
\begin{align}
&\min_{x} && c^T x\\
&\text{s.t.} && Ax \geq b\\
& && x \geq 0\\
& && x_i \in \mathbb{Z}, i \in I
\end{align}
$$

In [25]:
def run_cutting_planes(filepath):
    b = os.popen(filepath).read()
    #print(b)

def load_data(filepath):
    return json.load(open(filepath))

run_cutting_planes('/Users/tobiaskohler/Uni/CuttingPlanes/build-CuttingPlanes-Desktop_arm_darwin_generic_mach_o_64bit-Debug/CuttingPlanes')
d = load_data('test.json')
print(d)

# Access the system data
n, m = d['n'], d['m']
A = np.array(d['A'], dtype=np.float64)
b = np.array(d['b'], dtype=np.float64)
c = np.array(d['c'], dtype=np.float64)
I = np.array(d['I'], dtype=bool)
sols = d['sols']
n_cuts = b.shape[0] - m
assert(A.shape[0] == b.shape[0] and n == A.shape[1] and n == c.shape[0] and n_cuts==len(sols)-1)
assert(n == 2)

# Insert nonnegative constraints for visualisation
m += 2
A = np.vstack([[1,0], [0,1], A])
b = np.append([0,0], b)

{'A': [[-3.0, -2.0], [3.0, -2.0], [0.0, -1.0], [1.0, -1.0]], 'I': [True, True], 'b': [-6.0, -0.0, -1.0, 0.0], 'c': [0.0, -1.0], 'm': 2, 'n': 2, 'sols': [[1.0, 1.5], [0.6666666666666666, 1.0], [1.0, 1.0]]}


In [66]:
def plot_lp(c, A, b, sol=None, xlim=(0, 5), ylim=(0, 5)):

    ratio = (xlim[1]-xlim[0])/(ylim[1]-ylim[0])
    plt.figure(figsize=(10,10/ratio))
    assert(len(c) == 2)
    m = len(b)

    # Generate Meshgrid of Points
    x_vals = np.linspace(xlim[0], xlim[1], 400)
    y_vals = np.linspace(ylim[0], ylim[1], 400)
    X, Y = np.meshgrid(x_vals, y_vals)

    # Plot Objective Function
    Z = c[0]*X + c[1]*Y
    plt.imshow(Z, extent=[xlim[0], xlim[1], ylim[0], ylim[1]], origin='lower', cmap='viridis', alpha=0.3)

    # Plot Inequality Constraints
    for i in range(m):
        if A[i,1] != 0:
            plt.plot(x_vals, (b[i] - A[i,0]*x_vals) / A[i,1], 'g-', lw=2, label='C'+str(i))
        elif A[i,0] != 0:
            plt.axvline(x=b[i] / A[i, 0], color='g', linestyle='-', lw=2, label='C'+str(i))

    # Shade Feasible Region
    feasible_cmap = matplotlib.colors.ListedColormap(['white', 'lime'])
    feasible_region = np.all(A[:,0,None,None]*X + A[:,1,None,None]*Y >= b[:,None,None], axis=0)
    plt.contourf(X, Y, feasible_region, cmap=feasible_cmap, alpha=0.2)

    # Plot Integer Points
    x_int = np.arange(np.ceil(xlim[0]), np.floor(xlim[1])+1, 1)
    y_int = np.arange(np.ceil(ylim[0]), np.floor(ylim[1])+1, 1)
    X_int, Y_int = np.meshgrid(x_int, y_int)
    plt.scatter(X_int, Y_int, s=40, color='blue', label='Integers')

    # Plot Optimal Solution
    if sol is not None:
        plt.plot(sol[0],sol[1],'ro', markersize=10, label='x=(%g, %g)' % (sol[0],sol[1]))

    # Labels and Stuff
    plt.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    plt.xlabel('x_1')
    plt.ylabel('x_2')
    plt.title('Cutting Planes')
    plt.grid(color='gray', linestyle='-', linewidth=1)
    plt.xticks(np.arange(np.floor(xlim[0]), xlim[1], 1))
    plt.yticks(np.arange(np.floor(ylim[0]), ylim[1], 1))
    plt.xlim(xlim)
    plt.ylim(ylim)

    # Show plot
    plt.show()

In [68]:
vs = lineqs(A[:m,:], b[:m], show=False)
xlim, ylim = [-0, 1.1*np.max(vs[:,0])], [-0, 1.1*np.max(vs[:,1])]

def plot_cutting_planes(i):
    plot_lp(c, A[:m+i,:], b[:m+i], sol=sols[i], xlim=xlim, ylim=ylim)

interact(plot_cutting_planes, i=widgets.IntSlider(min=0, max=n_cuts, step=1, value=0, description='Iteration:'))

interactive(children=(IntSlider(value=0, description='Iteration:', max=2), Output()), _dom_classes=('widget-in…

<function __main__.plot_cutting_planes(i)>